> Martin Fridrich, 03/2022 

# Final project


In [1]:
# mount project dir
from google.colab import drive
drive.mount("/content/gdrive",  force_remount=True)
%cd gdrive/My Drive/Colab/
#!git clone  https://github.com/fridrichmrtn/hse-advanced-machine-learning
%cd hse-advanced-machine-learning
!git pull
%cd 02_competitive_data_science 
!pip install -r requirements.txt

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab
/content/gdrive/My Drive/Colab/hse-advanced-machine-learning
Updating 2a2e58f..38ce5af
error: Your local changes to the following files would be overwritten by merge:
	02_competitive_data_science/requirements.txt
	02_competitive_data_science/wk06/final_project.ipynb
Please commit your changes or stash them before you merge.
Aborting
/content/gdrive/My Drive/Colab/hse-advanced-machine-learning/02_competitive_data_science
     |████████████████████████████████| 86 kB 3.8 MB/s 
     |████████████████████████████████| 76.1 MB 69 kB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149866 sha256=7726d806045e330c2eaa158bb2cf27c66476dbc70620e51ebfacd56b2e757f9b
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [2]:
# set kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Personal/.kaggle"
# download data
!mkdir -p data/final_project
%cd data/final_project
!kaggle competitions download -c competitive-data-science-predict-future-sales
# unpack and remove zips
import os
import glob
for f in glob.glob('*.{}'.format("zip")):
  os.system("unzip "+f); os.system("rm "+f)
# get back
%cd /content/gdrive/MyDrive/Colab/hse-advanced-machine-learning/02_competitive_data_science

/content/gdrive/My Drive/Colab/hse-advanced-machine-learning/02_competitive_data_science/data/final_project
  0% 0.00/15.1M [00:00<?, ?B/s]
100% 15.1M/15.1M [00:00<00:00, 159MB/s]
/content/gdrive/MyDrive/Colab/hse-advanced-machine-learning/02_competitive_data_science


In [8]:
# lgbm on gpu
def check_lgbm_gpu():
  import numpy
  import lightgbm
  lightgbm.LGBMClassifier(device="gpu").fit(numpy.array([0,1]).reshape(-1,1),[1,0])
  print("Succes!")
try:
  print("Testing LGBM & GPU compliance...")
  check_lgbm_gpu()
except:
  print("Resolving the issues...")
  !pip uninstall -y lightgbm
  !pip install lightgbm --install-option=--gpu
  check_lgbm_gpu()

Testing LGBM & GPU compliance...
Resolving the issues...
Found existing installation: lightgbm 3.3.2
Uninstalling lightgbm-3.3.2:
  Successfully uninstalled lightgbm-3.3.2
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
  Using cached lightgbm-3.3.2.tar.gz (1.5 MB)
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


LightGBMError: ignored

## Data loading

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from itertools import product
import re
from fuzzywuzzy import fuzz

In [ ]:
DATA_DIR = "data/final_project/"
# sales
sales_train = pd.read_csv(DATA_DIR+"sales_train.csv")
sales_test = pd.read_csv(DATA_DIR+"test.csv")
# addional info
items = pd.read_csv(DATA_DIR+"items.csv")
item_categories = pd.read_csv(DATA_DIR+"item_categories.csv")
shops = pd.read_csv(DATA_DIR+"shops.csv")

# basic stats related to sales tab
sales_train.describe()

## Initial transformations

In [ ]:
# downcast dataframe
def optimize_numeric_dtypes(df):
    import pandas as pd
    float_cols = df.select_dtypes("float").columns
    int_cols = df.select_dtypes("integer").columns
    df[float_cols] = df[float_cols].\
        apply(pd.to_numeric, downcast="float")
    df[int_cols] = df[int_cols].\
        apply(pd.to_numeric, downcast="integer")
    return df

def optimize_categories(df):
    import pandas as pd
    cat_cols = df.select_dtypes("object").columns
    df[cat_cols] = df[cat_cols].\
        apply(lambda x: x.astype("category").cat.codes)
    return df    

In [ ]:
# duplicate shops
shop_duplicates_map = {0: 57, 1: 58, 11: 10, 40: 39}
sales_train["shop_id"] = sales_train["shop_id"].replace(shop_duplicates_map)
# datetime
sales_train["date"] = pd.to_datetime(sales_train["date"], format="%d.%m.%Y")
# filtering on test set
sales_train = sales_train.loc[sales_train.item_id.isin(sales_test["item_id"].unique()), :]
# drop training items with extreme or negative prices or sales counts
sales_train = sales_train[((sales_train["item_price"] > 0) & (sales_train["item_price"] <= 10000)) &
    ((sales_train["item_cnt_day"] > 0) & (sales_train["item_cnt_day"] <= 1000))]
# test-based cartesian product & inds
sales_index = pd.DataFrame(product(sales_test.item_id.unique(), sales_test.shop_id.unique(),
    range(35)),columns = ["item_id", "shop_id", "date_block_num"])
sales_index = optimize_numeric_dtypes(sales_index)
# sales table
sales_train["revenue"] = sales_train["item_price"]*sales_train["item_cnt_day"]
sales = sales_train.groupby(["shop_id", "item_id", "date_block_num"], as_index=False).agg(
    total_sold=("item_cnt_day", sum),  sum_revenue=("revenue",sum),
    n_transactions = ("item_cnt_day",pd.Series.count))\
        .sort_values("date_block_num")
sales = sales_index.merge(sales, how="left",
    on=["shop_id", "item_id", "date_block_num"])
sales["date_block_num"].fillna(34, inplace=True)
sales.loc[sales.date_block_num<34, ["total_sold", "sum_revenue", "n_transactions"]]=\
    sales.loc[sales.date_block_num<34, ["total_sold", "sum_revenue", "n_transactions"]].fillna(0)
sales = optimize_numeric_dtypes(sales)
del sales_train, sales_test, sales_index; gc.collect(); sales.head(3) 

## Exploration

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,4))
hist = sns.histplot(data=sales, x="total_sold", bins=10,
    palette=sns.color_palette("husl"))
hist.set_yscale("log")

In [ ]:
# top 5 items
sales.groupby("item_id")["total_sold"].sum().\
    sort_values(ascending=False).head(5)

In [ ]:
# targets per top 5 items over time
fig, ax = plt.subplots(1,1,figsize=(8,4))
sns.lineplot(data=sales[sales.item_id.isin([20949,2808,3732,17717,5822])].\
        groupby(["item_id","date_block_num"], as_index=False)["total_sold"].sum(),
    x="date_block_num", y="total_sold", hue="item_id",
    palette=sns.color_palette("husl", 5));    

In [ ]:
# top 5 shops
sales.groupby("shop_id")["total_sold"].sum().\
    sort_values(ascending=False).head(5)

In [ ]:
# targets per top 5 items over time
fig, ax = plt.subplots(1,1,figsize=(8,4))
sns.lineplot(data=sales[sales.shop_id.isin([31,25,28,42,57])].\
        groupby(["shop_id","date_block_num"], as_index=False)["total_sold"].sum(),
    x="date_block_num", y="total_sold", hue="shop_id",
    palette=sns.color_palette("husl", 5));    

In [ ]:
# top 5 categories
sales.merge(items).groupby("item_category_id")["total_sold"].sum().\
    sort_values(ascending=False).head(5)

In [ ]:
# price vs sales, demand almost inelastic
df = sales[sales.item_id.isin([20949,2808,3732,17717,5822])].\
        groupby(["item_id", "date_block_num"], as_index=False).\
            agg(sum_revenue=("sum_revenue",sum),
            total_sold=("total_sold",sum))
df["average_price"] = df.sum_revenue/df.total_sold
fig, ax = plt.subplots(1,1,figsize=(8,4))
scatter = sns.scatterplot(data=df, x="total_sold", y="average_price",
    hue="item_id", palette=sns.color_palette("husl", 5));
scatter.set_yscale("log"); scatter.set_xscale("log");

## Feature engineering

In [ ]:
# item groups based on adjacency and label similarity
items.item_name = items.item_name.apply(lambda x: re.sub("[^A-ZА-Яa-zа-я0-9 ]","", x))
items.item_name = items.item_name.apply(lambda x: re.sub("\\s+"," ", x))
item_names = pd.concat([items.item_id, items.item_name,
    items.item_id.shift().fillna(0), items.item_name.shift().fillna("")], axis=1)
item_names.columns = ["item_id", "item_name", "previous_item_id", "previous_item_name"]
item_names["similarity"] = item_names.apply(\
     lambda x: fuzz.ratio(x["item_name"],x["previous_item_name"]), axis=1)
item_names["group_start"] = item_names["similarity"] <= 75
item_group_map = item_names.loc[item_names.group_start, ["item_id"]]
item_group_map.columns = ["group_start"]
item_group_map["group_end"] = item_group_map["group_start"].shift(-1).\
    fillna(item_group_map["group_start"].max()+1).astype("int")
item_group_map["item_sim_id"] = list(range(len(item_group_map)))
# remap
items = items.merge(item_group_map, how="left",
    left_on=["item_id"], right_on=["group_start"])
items = items.sort_values("item_id")
items["item_sim_id"] = items["item_sim_id"].fillna(method="ffill").astype("int")
del item_names, item_group_map; gc.collect()
items = items[["item_id","item_name", "item_category_id", "item_sim_id"]]
# name length
items["item_name_len"] = items.item_name.apply(len).astype("int16")
items.head(3)

In [ ]:
# categories
cat_splits = item_categories.item_category_name.apply(lambda x: x.split(" - "))
item_categories = pd.concat([item_categories,
    pd.DataFrame([c if len(c)>1 else [c[0],""] for c in cat_splits],
        columns=["parent_cat","child_cat"])], axis=1)
items = items.merge(item_categories, how="inner")
# return categorical encoding
items = optimize_categories(items[["item_id", "item_category_id", "item_sim_id",
    "item_name_len", "parent_cat", "child_cat"]])
# add categories
sales = sales.merge(items, how="inner", on="item_id")
del item_categories, items; gc.collect(); sales.head(3)

In [ ]:
# date features
def working_days_count(from_date, to_date=None):
    import pandas as pd
    if to_date is None:
        to_date=from_date+pd.DateOffset(months=1)
    temp_date=from_date
    cnt=0
    while temp_date<to_date:
        if temp_date.weekday()<5:
            cnt+=1
        temp_date+=pd.DateOffset(days=1)
    return ((to_date-from_date).days, cnt, (to_date-from_date).days-cnt)
dates = pd.DataFrame(sales.date_block_num.unique(), columns=["date_block_num"])
dates["date"] = dates["date_block_num"].apply(lambda x: pd.to_datetime("01/01/2013")+\
    pd.DateOffset(months=x))
dates["year"] = dates.date.dt.year
dates["month"] = dates.date.dt.month
dates = optimize_numeric_dtypes(pd.concat([dates, pd.DataFrame.from_records(
        dates["date"].apply(lambda x: working_days_count(x)),
    columns=["total_days", "working_days", "weekend_days"])], axis=1))
sales = sales.merge(dates[["date_block_num","year","month","total_days", "working_days",
    "weekend_days"]], how="inner", on="date_block_num")
del dates; gc.collect(); sales.tail(3)    

In [ ]:
# pricing overall and inside cat
item_pricing = sales.groupby(["date_block_num","item_id","item_category_id"], as_index=False).\
    agg(total_sold=("total_sold", sum),  sum_revenue=("sum_revenue",sum))
train_ind = item_pricing.date_block_num<34
item_pricing.loc[train_ind,"average_price"] = item_pricing["sum_revenue"]/item_pricing["total_sold"]
item_pricing.loc[train_ind,"average_price"]  = item_pricing.groupby(["item_id"])["average_price"].\
    apply(lambda x: x.fillna(method="ffill").fillna(method="bfill"))
# add price changes
item_pricing.loc[train_ind,"price_change"] = item_pricing.groupby(["item_id"])["average_price"].\
    apply(lambda x: x/x.shift()).fillna(-1)
# add position within the category
item_pricing.loc[train_ind,"price_perc"] = item_pricing.groupby(["item_category_id", "date_block_num"])\
    ["average_price"].apply(lambda x: x.rank(pct=True)).fillna(-1)
item_pricing = optimize_numeric_dtypes(item_pricing[["date_block_num", "item_id", "price_change",
    "price_perc"]])
sales = sales.merge(item_pricing, how="left",
    on=["date_block_num", "item_id"])
del item_pricing; gc.collect(); sales.tail(3)

In [ ]:
# shop location
shops["city_id"] = shops.shop_name.apply(lambda x:\
    re.sub("[^A-ZА-Яa-zа-я0-9 ]","", x.split(" ")[0]))
shops = optimize_categories(shops[["shop_id", "city_id"]])
sales = sales.merge(shops, how="inner", on="shop_id")
del shops; gc.collect(); sales.tail(3)

In [ ]:
# time-series
# smoothed demand across various perspectives
train_ind = sales.date_block_num<34
# city
sales.loc[train_ind, "city_demand"] = sales[train_ind].\
    groupby(["city_id","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")
# shop
sales.loc[train_ind, "shop_demand"] = sales[train_ind].\
    groupby(["shop_id","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")
# categories
sales.loc[train_ind, "cat_demand0"] = sales[train_ind].\
    groupby(["parent_cat","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")
# categories
sales.loc[train_ind, "cat_demand1"] = sales[train_ind].\
    groupby(["child_cat","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")      
# categories
sales.loc[train_ind, "cat_demand2"] = sales[train_ind].\
    groupby(["item_sim_id","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")        
# item
sales.loc[train_ind, "item_demand"] = sales[train_ind].\
    groupby(["item_id","date_block_num"])["total_sold"].apply(lambda x:
        x.ewm(span=3, min_periods=3).mean()).fillna(0)#.astype("float32")
# item maturity
sales["item_maturity"] = (sales["date_block_num"] - sales[["item_id","date_block_num"]].merge(\
    sales[sales.n_transactions>0].groupby("item_id", as_index=False).\
        agg(date_block_num_min=("date_block_num",min)),
            how="left")["date_block_num_min"]).clip(-1,34).fillna(-1)
del train_ind; gc.collect();
sales = optimize_numeric_dtypes(sales)
sales.tail(3)

In [ ]:
lag_steps = [1,2,3]
lag_fill = {"total_sold":0, "sum_revenue":0, "n_transactions":0, "price_change":1, "price_perc":-1,
    "city_demand":0, "shop_demand":0, "cat_demand0":0,"cat_demand1":0,"cat_demand2":0, "item_demand":0}
for l in lag_steps:
    for c in lag_fill.keys():
        sales.loc[:,"lag"+str(l)+"_"+c] = sales.groupby([sales.shop_id,sales.item_id])\
            [c].shift(l).fillna(lag_fill[c])
#sales = sales[sales.date_block_num>25] # just reduce the dataset
target = "total_sold"
col_fil = [c for c in sales.columns if (c not in lag_fill.keys()) or (c ==target)]
sales = optimize_numeric_dtypes(sales.loc[:,col_fil])
sales.head(3)

In [ ]:
# col filters
target = "total_sold"
features = [c for c in sales.columns if (c not in [target])]
cat_cols = ["shop_id", "item_id", "date_block_num", "item_name_len",
    "item_category_id", "item_sim_id", "parent_cat", "child_cat",
    "year", "month", "total_days", "working_days", "weekend_days",
    "city_id"]
num_cols = [f for f in sales[features].columns if (f not in cat_cols) & (f not in [target])]
# target clippin
sales[target] = sales[target].clip(0,20)

In [ ]:
# ADD MEAN ENCODING
# ADD HYPEROPT

## Modeling pipeline

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, PolynomialFeatures
from category_encoders import TargetEncoder, PolynomialEncoder, CatBoostEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import IncrementalPCA
import lightgbm as lgbm
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression,ElasticNetCV,ElasticNet,BayesianRidge
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

ModuleNotFoundError: ignored

In [ ]:
# stage filters
b_ind = (sales.year==2015) & (sales.month==9) # 2nd stage
c_ind = (sales.year==2015) & (sales.month==10)
test_ind = (sales.year==2015) & (sales.month==11) # prediction
a_ind = ~b_ind & ~c_ind & ~test_ind # 1st stage

### First stage models

In [10]:
# lightgbm raw
lgbm_model = lgbm.LGBMRegressor(n_estimators=1000, learning_rate=.01,
  objective="rmse", device="gpu")
lgbm_model.fit(sales.loc[a_ind,features], sales.loc[a_ind,target],
  eval_set=(sales.loc[b_ind,features], sales.loc[b_ind,target]),
  callbacks=lgbm.early_stopping(stopping_rounds=10))

# pushout preds
lgbm_b_feature = lgbm_model.predict(sales.loc[b_ind,features]).clip(0,20)
lgbm_c_feature = lgbm_model.predict(sales.loc[c_ind,features]).clip(0,20)
lgbm_test_feature = lgbm_model.predict(sales.loc[test_ind,features]).clip(0,20)

NameError: ignored

In [ ]:
# catboost on raw data
cat_model = CatBoostRegressor(cat_features=cat_cols,
    verbose=100, task_type="GPU")
cat_model.fit(sales.loc[a_ind,features], sales.loc[a_ind,target])

# pushout preds
cat_b_feature = cat_model.predict(sales.loc[b_ind,features]).clip(0,20)
cat_c_feature = cat_model.predict(sales.loc[c_ind,features]).clip(0,20)
cat_test_feature = cat_model.predict(sales.loc[test_ind,features]).clip(0,20)

Learning rate set to 0.128477
0:	learn: 0.9846387	total: 744ms	remaining: 12m 23s
100:	learn: 0.5948615	total: 1m 27s	remaining: 13m 2s
200:	learn: 0.5653991	total: 2m 55s	remaining: 11m 36s
300:	learn: 0.5512966	total: 4m 21s	remaining: 10m 6s
400:	learn: 0.5423187	total: 5m 46s	remaining: 8m 38s
500:	learn: 0.5360904	total: 7m 14s	remaining: 7m 13s
600:	learn: 0.5305284	total: 8m 40s	remaining: 5m 45s
700:	learn: 0.5258658	total: 10m 7s	remaining: 4m 19s
800:	learn: 0.5210036	total: 11m 34s	remaining: 2m 52s
900:	learn: 0.5180560	total: 12m 59s	remaining: 1m 25s
999:	learn: 0.5155353	total: 14m 24s	remaining: 0us


In [ ]:
# lightgbm raw
lgbm_model = lgbm_model = Pipeline(steps=[("ct",ColumnTransformer(transformers=[
        ("se", StandardScaler(), num_cols),
        ("te", TargetEncoder(cols=cat_cols, drop_invariant=True,
            min_samples_leaf=5*10**4, smoothing=10**3), cat_cols)])),
    ("lgbm", lgbm.LGBMRegressor(n_estimators=1000, learning_rate=.01,
        max_depth=4, subsample=.5, num_leaves = 50, device="gpu"))])
lgbm_model.fit(sales.loc[a_ind,features], sales.loc[a_ind,target])

# pushout preds
lgbm_b_feature = lgbm_model.predict(sales.loc[b_ind,features]).clip(0,20)
lgbm_c_feature = lgbm_model.predict(sales.loc[c_ind,features]).clip(0,20)
lgbm_test_feature = lgbm_model.predict(sales.loc[test_ind,features]).clip(0,20)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7472
[LightGBM] [Info] Number of data points in the train set: 6854400, number of used features: 44
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 27 dense feature groups (183.03 MB) transferred to GPU in 0.181261 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 0.183869
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

In [ ]:
# lightgbm pca
lgbmp_model = lgbm_model = Pipeline(steps=[("ct",ColumnTransformer(transformers=[
        ("se", StandardScaler(), num_cols),
        ("te", TargetEncoder(cols=cat_cols, drop_invariant=True,
            min_samples_leaf=5*10**4, smoothing=10**3), cat_cols)])),
    ("qe", QuantileTransformer()),
    ("pca", IncrementalPCA(n_components=20)),
    ("lgbm", LGBMRegressor(n_estimators=1000, learning_rate=.01,
        max_depth=4, subsample=.5, num_leaves = 50, device="gpu"))])
lgbmp_model.fit(sales.loc[a_ind,features], sales.loc[a_ind,target])

# pushout preds
lgbmp_b_feature = lgbmp_model.predict(sales.loc[b_ind,features]).clip(0,20)
lgbmp_c_feature = lgbmp_model.predict(sales.loc[c_ind,features]).clip(0,20)
lgbmp_test_feature = lgbmp_model.predict(sales.loc[test_ind,features]).clip(0,20)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5100
[LightGBM] [Info] Number of data points in the train set: 6854400, number of used features: 20
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 20 dense feature groups (130.74 MB) transferred to GPU in 0.150212 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.183869
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

In [ ]:
# lightgbm clf
lgbmc_model = lgbm_model = Pipeline(steps=[("ct",ColumnTransformer(transformers=[
        ("se", StandardScaler(), num_cols),
        ("te", TargetEncoder(cols=cat_cols, drop_invariant=True,
            min_samples_leaf=5*10**4, smoothing=10**3), cat_cols)])),
    #("qe", QuantileTransformer()),
    #("pca", IncrementalPCA(n_components=10)),
    ("lgbm", LGBMClassifier(objective="multiclass", n_estimators=1000, learning_rate=.01,
        max_depth=4, subsample=.5, num_leaves=150, device="cpu"))])
#clf_target = (sales[target]>1).astype("int")
clf_target = (sales.loc[a_ind,target].rank(pct=True).round(1)*10).astype("int")
lgbmc_model.fit(sales.loc[a_ind,features], clf_target[a_ind])

# pushout preds
lgbmc_b_feature = lgbmc_model.predict(sales.loc[b_ind,features])
lgbmc_c_feature = lgbmc_model.predict(sales.loc[c_ind,features])
lgbmc_test_feature = lgbmc_model.predict(sales.loc[test_ind,features])

[LightGBM] [Info] Number of positive: 221847, number of negative: 6632553
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.900717 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7371
[LightGBM] [Info] Number of data points in the train set: 6854400, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.032366 -> initscore=-3.397757
[LightGBM] [Info] Start training from score -3.397757
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

### Second stage model

In [ ]:
b_features = np.stack([cat_b_feature, lgbm_b_feature,
    lgbmp_b_feature, lgbmc_b_feature], axis=1)
c_features = np.stack([cat_c_feature, lgbm_c_feature,
    lgbmp_c_feature, lgbmc_c_feature], axis=1)
test_features = np.stack([cat_test_feature, lgbm_test_feature,
    lgbmc_test_feature, lgbmp_test_feature], axis=1)

lr_model = LinearRegression()
lr_model.fit(b_features, sales.loc[b_ind,target])

train_rsme = np.mean((lr_model.predict(b_features).clip(0,20)-sales.loc[b_ind,target])**2)**(1/2)
val_rsme = np.mean((lr_model.predict(c_features).clip(0,20)-sales.loc[c_ind,target])**2)**(1/2)
print("Stacked pipeline> RMSE on train set: {}; on val set: {}".format(train_rsme, val_rsme))

Stacked pipeline> RMSE on train set: 0.8194030663226025; on val set: 0.8473648970050974


In [ ]:
test_mapping = pd.read_csv(DATA_DIR+"test.csv")
test_predictions = sales.loc[test_ind,["item_id", "shop_id"]]
test_predictions["item_cnt_month"] = lr_model.predict(test_features).clip(0,20)
test_predictions = test_mapping.merge(test_predictions, on=["item_id", "shop_id"],how="left").fillna(0)
test_predictions[["ID", "item_cnt_month"]].to_csv("data/submission.csv", index=False)

## Submission

In [ ]:
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f data/submission.csv -m "Meh, but..."

100% 4.30M/4.30M [00:00<00:00, 14.2MB/s]
400 - Bad Request
